# Baixando os dados

Para facilitar o processamento e uso das ferramentas de análise e visualização escolhi baixar tudo que é preciso primeiro. Por meio das APIs oficias da Riot Games é possível recuperar informações das partidas, assim como suas respectivas linhas do tempo. Essas informações são salvas em duas pastas em um json por partida.

APIs Utilizadas:  
SUMMONER-V4 (utilizada no próprio site)  
ACCOUNT-V1 (utilizada no próprio site)   
MATCH-V5

In [1]:
import requests
import pandas as pd
import numpy as np
import altair as alt
import json
from pathlib import Path
from datetime import datetime
import time
from tqdm.notebook import trange, tqdm

In [2]:
with open("apikey.txt", "rb") as f: 
    apikey = f.read().decode("utf-8")
with open("ODESAFIO.json","rb") as f: 
    puuid = json.loads(f.read())["puuid"]

In [3]:
rate_time = .85 # ~(50/60)

Muitas limitações foram encontradas nesse contexto usando o `datetime` do próprio python, então foi preciso trocar para o gerenciamento de tempo do `pandas`. 

Muitos erros aconteceram utilizando o parâmetro `startTime` na URL para conseguir o ID das partidas, então para conseguir iniciar o trabalho de preprocessamento eu preferi usar somente a data final do desafio. Eventualmente isso deve ser corrigido para incluir precisamente o período.

__corrigido__: Dois erros, um seguido do outro. A documentação especificava segundos, e o argumento estava em milisegundos, o `endTime` funcionava por que o tempo era no futuro (segundos esperados * 100 dos milisegundos que eu passava como argumento), mas o `starTime` também indicava o futuro. Em seguida, o formato de `startime` estava escrito como _dia/mes/ano_ assim como o `endTime` no `pandas`. Em `endTime` era compreendido já que o dia passava de 12, então não tinha como ser um mês, mas no `startTime` o formato era lido como _mes/dia/ano_, já que o dia era menor que 12. 

In [4]:
matches_id = []

startTime = int(pd.Timestamp('2022-03-10').timestamp()) # 11/03
endTime = int(pd.Timestamp('2022-03-16').timestamp()) # 14/03

# pd.to_datetime(startTime, unit="s"), pd.to_datetime(endTime, unit="s")

matches_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?startTime={startTime}&endTime={endTime}&start=0&count=100&api_key={apikey}"
r = requests.get(matches_url)
matches_id = r.json()

In [5]:
print(matches_id, len(matches_id))

['BR1_2481071668', 'BR1_2481041619', 'BR1_2481060618', 'BR1_2481020856', 'BR1_2481017412', 'BR1_2480995664', 'BR1_2480964299', 'BR1_2480949667', 'BR1_2480917117', 'BR1_2480922193', 'BR1_2480857504', 'BR1_2480862080', 'BR1_2480826690', 'BR1_2480794279', 'BR1_2480738971', 'BR1_2480724535', 'BR1_2480710991', 'BR1_2480664893', 'BR1_2480632805', 'BR1_2480497171', 'BR1_2480503980', 'BR1_2480481707', 'BR1_2480429560', 'BR1_2480446530', 'BR1_2480415103', 'BR1_2480442152', 'BR1_2480431260', 'BR1_2480399154', 'BR1_2480388413', 'BR1_2480378001', 'BR1_2480348399', 'BR1_2480367158', 'BR1_2480347482', 'BR1_2480356476', 'BR1_2480384739', 'BR1_2480383688', 'BR1_2480392117', 'BR1_2480371314', 'BR1_2480308850', 'BR1_2480296325', 'BR1_2480322557', 'BR1_2480280347', 'BR1_2480274280', 'BR1_2480220562', 'BR1_2480208053', 'BR1_2480213769', 'BR1_2480191626', 'BR1_2480157938', 'BR1_2480153510', 'BR1_2480121325', 'BR1_2480098848', 'BR1_2480065470', 'BR1_2480090708', 'BR1_2479999393', 'BR1_2479994762', 'BR1_2479

Os laços abaixo só usam o id das partidas para recuperar a informação das partidas e linha do tempo.

In [6]:
for match_id in tqdm(matches_id):
    match_data_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={apikey}"
    r = requests.get(match_data_url)
    # time.sleep(rate_time)
    with open(f"matches/{match_id}.json", "w") as outfile:
        json.dump(r.json(), outfile, indent=3, sort_keys=True)

  0%|          | 0/93 [00:00<?, ?it/s]

In [7]:
for match_id in tqdm(matches_id):
    match_data_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline?api_key={apikey}"
    r = requests.get(match_data_url)
    # time.sleep(rate_time)
    with open(f"timelines/{match_id}.json", "w") as outfile:
        json.dump(r.json(), outfile, indent=3, sort_keys=True)

  0%|          | 0/93 [00:00<?, ?it/s]

Por enquanto os downloads terminam aqui, mas provavelmente um acesso dinâmico vai ser necessário na API ACCOUNT-V1 ou SUMMONER-V4 para recuperar os nomes dos outros participantes das partidas se for necessário.